In [1]:
import numpy as np

In [2]:
feature = np.load('num1.npy')
target = np.load('target.npy')

In [6]:
target.shape, feature.shape

((431424, 3), (431424, 48, 48, 3))

In [7]:
from keras.applications import VGG16
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Layer, Dense, Flatten, Dropout, Activation

### 전이학습

In [ ]:
transfer_model = VGG16(weights='imagenet', include_top = False, input_shape=(48,48,3))
transfer_model.trainable = False

In [ ]:
finetune_model = Sequential()
finetune_model.add(transfer_model)
finetune_model.add(Flatten())
finetune_model.add(Dense(64, activation='relu'))
finetune_model.add(Dropout(0.5))
finetune_model.add(Dense(3, activation = 'softmax'))

In [ ]:
finetune_model.summary()

In [ ]:
finetune_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience = 5, min_delta = 0.05)

In [ ]:
result = finetune_model.fit(new_img, new_label,epochs = 5,batch_size=1000,
                           validation_split=0.2,
                           callbacks=[early_stop])

In [19]:
from hyperopt import fmin, tpe, hp, Trials
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam


# 함수로서의 학습 및 평가 로직 정의
def train_and_evaluate(params):
    # 하이퍼파라미터 설정
    learning_rate = params['learning_rate']
    dropout_rate = params['dropout_rate']

    # 모델 구성
    transfer_model = VGG16(weights='imagenet', include_top=False, input_shape=(48, 48, 3))
    transfer_model.trainable = False
    finetune_model = Sequential()
    finetune_model.add(transfer_model)
    finetune_model.add(Flatten())
    finetune_model.add(Dense(64, activation='relu'))
    finetune_model.add(Dropout(dropout_rate))
    finetune_model.add(Dense(3, activation='softmax'))
    finetune_model.compile(optimizer=Adam(learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])

    # 조기 종료 콜백 설정
    early_stop = EarlyStopping(monitor='val_loss', patience=5, min_delta=0.05)

    # 모델 학습
    result = finetune_model.fit(feature, target, epochs=5, batch_size=1000,
                                validation_split=0.2,
                                callbacks=[early_stop])

    # 최종 검증 정확도 반환
    val_accuracy = result.history['val_accuracy'][-1]
    return -val_accuracy  # 목적 함수는 정확도를 최대화해야 하므로 음수로 반환

In [ ]:

# 하이퍼파라미터 탐색 공간 정의
space = {
    'learning_rate': hp.uniform('learning_rate', 0.0001, 0.1),
    'dropout_rate': hp.uniform('dropout_rate', 0.2, 0.6),
#    'batch_size': hp.choice('batch_size', [500, 1000])
#    'num_units_hidden_layer': hp.choice('num_units_hidden_layer', [100])
}

# Hyperopt를 사용하여 최적의 하이퍼파라미터 찾기
trials = Trials()
best = fmin(fn=train_and_evaluate, space=space, algo=tpe.suggest, max_evals=3, trials=trials)

# 최적의 하이퍼파라미터 출력
print("Best Hyperparameters:", best)


Epoch 1/5                                                                                                              

346/346 [==============================] - 3164s 9s/step - loss: 1.2240 - accuracy: 0.3325 - val_loss: 1.1001 - val_accuracy: 0.3278

Epoch 2/5                                                                                                              

346/346 [==============================] - 2902s 8s/step - loss: 1.0991 - accuracy: 0.3351 - val_loss: 1.0993 - val_accuracy: 0.3361

Epoch 3/5                                                                                                              

346/346 [==============================] - 2801s 8s/step - loss: 1.0994 - accuracy: 0.3319 - val_loss: 1.0996 - val_accuracy: 0.3278

Epoch 4/5                                                                                                              

346/346 [==============================] - 2912s 8s/step - loss: 1.0992 - accuracy: 0.3328 - val_loss: 1.0994 - va